# Extracting files from :

    * Eurostat [Population]
    
    * ECAD     [Températures]
    
    ==> These 2 types of files were merged by upstream processing


# Libraries and functions

In [1]:
###########################################################################################################################################
##############################                                                                               ##############################
##############################                             European Climate Assessment  &                    ##############################
##############################                                     Eurostat                                  ##############################
##############################                                 Updated 14/11/22                              ##############################
###########################################################################################################################################

###  librairies declaration
import warnings
import time
import pandas                             as pd
#import duckdb                             as db
import missingno                          as missin

import numpy                              as np
lasource_de_donnees="Sources : European Climate Assessment & Dataset (ECAD) & Eurostat urb-cpop - Updated 14/11/22"
import plotly.express                     as px
import plotly.graph_objects               as go

from math import *
import scipy.stats                        as st

# missing value
import missingno                          as msno # msno.bar(le_df) ou matrix(le_df) ou heatmap(le_df)

from scipy.stats                          import chi2_contingency
from sklearn.cluster                      import KMeans
import seaborn                            as sns; sns.set()

import matplotlib.pyplot                  as plt

import statsmodels.api                    as sm
import statsmodels.formula.api            as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn                              import preprocessing,cluster,decomposition,metrics  # normalisation

# Pour la fonction biplot :
import matplotlib                         as mpl
import matplotlib.cm                      as cm
from scipy.spatial                        import ConvexHull

# Normalize data :
from sklearn.preprocessing                import StandardScaler

warnings.filterwarnings('ignore')

# Set max rows displayed for readability
#pd.set_option("display.max_rows", 23)
# Configure display options
pd.set_option("display.max_columns", None)

strdirectory="../Analyse Temperature Europe/"

In [2]:
###########################################################################################################################################
##############################                                                                               ##############################
##############################                             Definition of functions                          ##############################
##############################                                                                               ##############################
###########################################################################################################################################

def conversion_sexadecimale_en_decimal_degre(la_list):
  """
    Geographic coordinates are often given in degrees (1/90 of a right angle), minutes of arc (1/60 of a degree), and seconds of arc
    (1/60 of an arc minute), which is not a problem for computers that work in binary. However, computer scientists judge
    sometimes the sexagesimal system is impractical to manipulate and, without going so far as to use the grades (the grade being 1/100 of a right angle),
    prefer to convert minutes and seconds into decimal fractions of degrees (we commonly use in this case the term "decimal degrees", at the risk of 
    confusion with the ranks). General wording :
    latitude (decimal degrees) = degrees + (minutes / 60) + (seconds / 3600)

    This function :
     latitude = sum(float(x) / 60 ** n for n, x in eGeographic coordinates are often given in degrees (1/90 of a right angle), minutes of arc (1/60 of a degree), and seconds of bow
    (1/60 of an arc minute), which is not a problem for computers that work in binary. However, computer scientists judge
    sometimes the sexagesimal system is impractical to manipulate and, without going so far as to use the grades (the grade being 1/100 of a right angle),
    prefer to convert minutes and seconds into decimal fractions of degrees (we commonly use in this case the term "decimal degreesnumerate(latitude[:-1].split('-'))) * (1 if 'N' in latitude[- 1] else -1)
      longitude = sum(float(x) / 60 ** n for n, x in enumerate(longitude[:-1].split('-'))) * (1 if 'E' in longitude[-1] else - 1)
  """
  l_row=pd.Series(la_list).iloc[0]
  print("Element 0 :",l_row[0],"Element 1 :",l_row[1],"Element 2 :",l_row[2])
  #degree=pd.to_numeric(l_row[0]) #; minute =la_list[1].astype(int) ; seconde = la_list[2].astype(int)
  la_valeur=pd.to_numeric(l_row[0],downcast='signed') + (pd.to_numeric(l_row[1],downcast='signed')/60) + (pd.to_numeric(l_row[2],downcast='signed')/3600)
  print("La conversion donne :",la_valeur)
  return la_valeur


def conversion_sexadecimale_en_decimal_degre_2(l_row):
  ''' This allows you to convert from decimal to degree
  '''
  la_valeur=pd.to_numeric(l_row[0],downcast='signed') + (pd.to_numeric(l_row[1],downcast='signed')/60) + (pd.to_numeric(l_row[2],downcast='signed')/3600)
  return round(la_valeur,4)

def attribution_classe(x,df_classe_maitre):
  ''' This allows you to assign a class to the data read.
  '''  
  index_classe=0
  for index, row in df_classe_maitre.iterrows():
    if (x > row['borne_inf_strict'] and x <= row['borne_sup_egale']):
      index_classe=row['la_borne']
  return index_classe

def Saison_identifie(str_m,str_d):
  ''' allows you to assign the same rule for season detection for all countries
  '''    
  val=""
  if (str_m=='12' or str_m=='01' or str_m=='02'):
    val="hiver"
  if (str_m=='03' or str_m=='04' or str_m=='05'):
    val="printemps"
  if (str_m=='06' or str_m=='07' or str_m=='08'):
    val="ete"
  if (str_m=='09' or str_m=='10' or str_m=='11'):
    val="automne"
  return val

# Declaration of dictionaries

Création de dictionnaire afin d'identifier le pays et de positionner son nom ainsi que chaque capitale

In [3]:
##   Insertion of country names for our study; this facilitates our understanding of graphs.
les_noms={}
les_noms["FR"] = "France";les_noms["GB"] = "Royaume-Uni";les_noms["IT"] = "Italie";les_noms["LV"] = "Lettonie"
les_noms["EE"] = "Estonie";les_noms["LT"] = "Lituanie";les_noms["NL"] = "Pays-Bas";les_noms["HU"] = "Hongrie"
les_noms["SE"] = "Suede" ;les_noms["NO"] = "Norvege";les_noms["RO"] = "Roumanie"
les_noms["PL"] = "Pologne" ;les_noms["ES"] = "Espagne";les_noms["AT"] = "Autriche" ;les_noms["DE"] = "Allemagne"
les_noms["TR"] = "Turquie"

les_capitales={}
les_capitales["PARIS"]="O";les_capitales["LONDON"]="O"
les_capitales["RIGA"]="O";les_capitales["TALLINN"]="O"
les_capitales["VILNIUS"]="O";les_capitales["AMSTERDAM"]="O"
les_capitales["BUDAPEST"]="O";les_capitales["STOCKHOLM"]="O"
les_capitales["OSLO"]="O";les_capitales["WARSZAWA"]="O"
les_capitales["MADRID"]="O";les_capitales["WIEN"]="O"
les_capitales["BERLIN"]="O"

# Merge files

Merge files 

In [4]:

# Start of extraction
debut=time.time()

# Initialize an empty DataFrame to store the concatenation
df_fichier = pd.DataFrame()

# Loop to read each file and concatenate its contents to the main DataFrame
for cle in les_noms:
    #print("Pays : ",cle)
    df_temporaire = pd.read_excel(strdirectory+"DecennieSourceWeather_ce_Pays_"+cle+".xlsx")
    df_fichier = pd.concat([df_fichier, df_temporaire], ignore_index=True)
    
fin=time.time()
min=(fin-debut)//60
sec=(fin-debut)-(min*60)
print("---> Durée de traitement de chargement fichier :",min,"min",sec,"sec")



---> Durée de traitement de chargement fichier : 8.0 min 7.362792253494263 sec


# Pre processing

PRE PROCESSING - ETAPE 1  :
    
    Date format
    
    Rename
    

In [5]:
# Change date formatChange date format
df_fichier['DATE_dt']=pd.to_datetime(df_fichier['DATE'],format="%Y%m%d")
#fichier['YEAR_dt']=fichier['DATE_dt'].dt.year
df_fichier['YEAR_dt']=pd.to_datetime(df_fichier['DATE_dt'],format="%Y")
# recovery of years and months
df_fichier['DATE_str_YEAR']=df_fichier['DATE'].astype(str).str[0:4]
df_fichier['DATE_str_MONTH']=df_fichier['DATE'].astype(str).str[4:6]
df_fichier['DATE_str_DAY']=df_fichier['DATE'].astype(str).str[6:8]
## Correct the temperature value
df_fichier['TEMPERATURE']=df_fichier['TX']*10**-1

df_fichier.rename(columns={'Value_corrigee':'NB_HAB_BRUT','sans_virgule_corrigee':'NB_HAB','CITIES_clean':'CITY'},inplace=True)

PRE PROCESSING - ETAPE 2  :
   
    Interesting column

In [6]:
# Column with uninteresting data for the rest of the analysis; to delete :
df_fichier.drop(['PARNAME','TX','DATE','Q_TX','TIME','STAID_num','SOUID','NB_HAB_BRUT'], axis=1, inplace=True)


PRE PROCESSING - ETAPE 3 :
    
    Filter on data > 2002 ?
    

In [7]:
# Data recovery from 01/01/2002; LOWER TERMINAL FOR DATES
'''
df_fichier=df_fichier.query("YEAR_dt > 2002")
'''
# Identifying the seasons
df_fichier['Saison']=df_fichier.apply(lambda row : Saison_identifie(row['DATE_str_MONTH'],row['DATE_str_DAY']), axis = 1)
#------------------------------------------------------------------------------------------------------------------------***
# Conservation of the global file before processing
df_fichier_brut=df_fichier

PRE PROCESSING - ETAPE 4 :

    Création de classe pour identifier les saisons

In [8]:
# Identifying the seasons
# df_fichier['Saison']=df_fichier.apply(lambda row : Saison_identifie(row['DATE_str_MONTH'],row['DATE_str_DAY']), axis = 1)

#Conservation du fichier global avant traitement
#df_fichier_brut=df_fichier

PRE PROCESSING - ETAPE 5 :

    LAT, LON

In [9]:
# Creating a column that breaks down latitude and longitude data
df_fichier['LAT_Decompose']=df_fichier['LAT'].str.split(":")
df_fichier['LON_Decompose']=df_fichier['LON'].str.split(":")

deb=time.time()

print("Conversion en degré décimal 1/")
# Objective of this block is to transform all the hexadecimal references into decimal degrees
# Then insert them into a dictionary in order to read it to assign the correct transformation to the main file
df_lon=df_fichier.groupby(["LON"],as_index=False).agg({'CN':'count'})
df_lat=df_fichier.groupby(["LAT"],as_index=False).agg({'CN':'count'})
# We give the same name to the column to prepare for concatenation
df_lon.rename(columns={'LON':'lat_lon'},inplace=True)
df_lat.rename(columns={'LAT':'lat_lon'},inplace=True)
# Concatenation of values
df_lat_lon=pd.concat([df_lat,df_lon],ignore_index=True)
# uniqueness values
df_lat_lon.groupby(["lat_lon"],as_index=False).agg({'CN':'count'})
df_lat_lon['lat_lon_Decompose']=df_lat_lon['lat_lon'].str.split(":")
df_lat_lon.drop(columns={'CN'},inplace=True)
df_lat_lon['lat_lon_degre']=df_lat_lon.apply(lambda row : conversion_sexadecimale_en_decimal_degre_2(row['lat_lon_Decompose']),axis=1)

# dictionary to save data frames
le_dict={}

for i in range(len(df_lat_lon)):
  # Adding elements of a dictionary with le_dict[the key] = the value
  le_dict[df_lat_lon.iloc[i,0]] = df_lat_lon.iloc[i,2]

# We reconcile the data from the dictionaries and the df :
# Data initialization
lat=[];lon=[];noms=[];capitales=[]

lat_cle='00:00:00';lon_cle='00:00:00';lat_val=0;lon_val=0
cn_cle='';cn_val='inconnu';nom_cap='inconnu';nom_cap_cle=''
for i in range(len(df_fichier)):
  nom_cap_val='N'
  if (lat_val ==0 and lon_val==0 and cn_val=='inconnu') or lat_cle!=df_fichier.iloc[i,5] or lon_cle!=df_fichier.iloc[i,6] :
    lat_cle=df_fichier.iloc[i,3]; lon_cle=df_fichier.iloc[i,4] ; cn_cle=df_fichier.iloc[i,2]  ;nom_cap_cle=df_fichier.iloc[i,0]
    for k in le_dict.keys():
      if k==lat_cle:
        lat_val=le_dict.get(k)
      if k==lon_cle:
        lon_val=le_dict.get(k)
    for j in les_noms.keys():
        if j==cn_cle:
          cn_val=les_noms.get(j)
    for l in les_capitales.keys():
        if l==nom_cap_cle:
          nom_cap_val=les_capitales.get(l)

  lat.append(lat_val)
  lon.append(lon_val)
  noms.append(cn_val)
  capitales.append(nom_cap_val)

# We add the columns found to the df
df_fichier['LAT_degre']=lat
df_fichier['LON_degre']=lon
df_fichier['CN_NAME']=noms
df_fichier['CAPITALE']=capitales

# Conservation of the global file before processing
df_fichier_brut_convertit=df_fichier.copy(deep=True)

# We clean the uninteresting columns
df_fichier.drop(['LAT_Decompose','LON_Decompose'], axis=1, inplace=True)

fin=time.time()
min=(fin-debut)//60
sec=(fin-debut)-(min*60)
print("---> Conversion processing time :",min,"min",sec,"sec")


Conversion en degré décimal 1/
---> Conversion processing time : 15.0 min 30.821916580200195 sec


PRE PROCESSING - ETAPE 6 :

    Removing duplicate cities
    

In [31]:
# Eviter les doublons sur même ville
# A city must appear 1 time PER DAY OF TEMPERATURE TAKING, even if it has several sensors!!
# It is therefore necessary to average the heights of the sensors, the temperatures, the latitude, the longitude and the number of inhabitants
print("Removing duplicate cities (pls sensors) 2/")

fichier_LAT_LON_HGHT=df_fichier.groupby(['CITY','CN_NAME','CN','CAPITALE'],as_index=False).agg({'LAT_degre':'mean','LON_degre':'mean','HGHT_num':'mean'})
#fichier_LON=df_fichier.groupby(['CITY','CN_NAME','CN','CAPITALE'],as_index=False).agg({'LON_degre':'mean'})
#fichier_HGHT=df_fichier.groupby(['CITY','CN_NAME','CN','CAPITALE'],as_index=False).agg({'HGHT_num':'mean'})

#fichier_LAT_LON=pd.merge(fichier_LAT,fichier_LON,on=['CITY','CN_NAME','CN','CAPITALE'],how='inn,er')
#fichier_LAT_LON_HGHT=pd.merge(fichier_LAT_LON,fichier_HGHT,on=['CITY','CN_NAME','CN','CAPITALE'],how='inner')
#fichier_LAT_LON_HGHT
fichier_prep=pd.merge(df_fichier,fichier_LAT_LON_HGHT,on=['CITY','CN_NAME','CN','CAPITALE'],how='inner')

fichier_prep.drop(['HGHT_num_x','LAT_degre_x','LON_degre_x'], axis=1, inplace=True)

fichier_prep.rename(columns={'LAT_degre_y':'LAT_degre','LON_degre_y':'LON_degre','HGHT_num_y':'HGHT_num'},inplace=True)
# Réorganiser les colonnes dans l'ordre souhaité
nouvel_ordre_colonnes = ['CITY','CN', 'CN_NAME','CAPITALE','LAT','LON','Saison','LAT_degre','LON_degre','HGHT_num',
                        'DATE_dt','YEAR_dt','DATE_str_YEAR','DATE_str_MONTH','DATE_str_DAY','TEMPERATURE', 'NB_HAB']
fichier_prep=fichier_prep[nouvel_ordre_colonnes]

fichier=fichier_prep.groupby(["CITY","CN_NAME","CN","CAPITALE","Saison","DATE_dt","YEAR_dt","DATE_str_YEAR",
        'DATE_str_MONTH','DATE_str_DAY'],as_index=False).agg({'LAT_degre':'mean',
        'LON_degre':'mean','HGHT_num':'mean',
        'TEMPERATURE':'mean','NB_HAB':'mean'})



Removing duplicate cities (pls sensors) 2/


PRE PROCESSING - ETAPE 7 :

    Delete observations if no density of inhabitants
    Rounded figures on LAT and LON

In [32]:
# Objectif : retirer les lignes pour lesquelles le nombre d'habitants est null
print("Suppression des lignes pour lesquelles pas de données densité d'habitant 3/")
# Identifies the lines for which the number of inhabitants is not specified
df_city_cn_avec_hab_null=fichier.loc[fichier['NB_HAB'].isnull()].groupby(["CITY","CN",'CN_NAME'],as_index=False).agg({'DATE_dt':'count'})
df_city_cn_avec_hab_null['flag']='O'
# Merge into left inner join using the flag column, I delete the left join result and I eliminate my lines having a zero number of inhabitants
tempo=fichier.merge(df_city_cn_avec_hab_null,on=['CITY','CN','CN_NAME'] ,how='left').query("flag != 'O'")
tempo.drop(['flag','DATE_dt_y'], axis=1,inplace=True)
tempo.rename(columns={'DATE_dt_x':'DATE_dt'},inplace=True)

# We round so as not to have different lat and lon
fichier['LAT_degre']=fichier['LAT_degre'].round(5)
fichier['LON_degre']=fichier['LON_degre'].round(5)
print("End of 1st part of treatment")

Suppression des lignes pour lesquelles pas de données densité d'habitant 3/
End of 1st part of treatment


In [33]:
#######################################                 ECRITURE                #################################

In [34]:
fichier.to_csv(strdirectory+'Extraction_etape_1_data_temps_hab.csv', index=False,encoding='utf-8')

In [42]:
#######################################                     FIN                 ################################